# RewardFunction13
```
Key Elements:
1. Uses vector curvature calculations
   - If cos(theta) is closer to 1, then the track is straight
   - And if the waypoints are closer together, assume that the track is curved
```

In [2]:
def reward_function(params):
    
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    is_left_of_center = params['is_left_of_center']
    track_width = params['track_width']
    progress = params['progress']/100
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params["steps"]
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 300
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.6 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete.  if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    else:        
        reward = REWARD_MAX * max(progress, 0.4)
        
    # Determine waypoints
    way0 = waypoints[closest_waypoints[0]] # closest waypoint behind car
    way1 = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    way2 = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    vector1 = [way1[0] - way0[0], way1[1] - way0[1]]
    vector2 = [way2[0] - way1[0], way2[1] - way1[1]]
    
    dist1 = (vector1[0]**2 + vector1[1]**2)**0.5    
    dist2 = (vector2[0]**2 + vector2[1]**2)**0.5 
    
    trackCurved = False
    if (dist2 > 0 and dist1 > 0):
        #we will analyze the relative distances between waypoints
        #compute the cosine of the angle between the vectors connecting consecutive waypoints
        cosineAngle = abs(vector1[0]*vector2[0] + vector1[1]*vector2[1]) / (dist1 * dist2)
        #cosineAngle will be closer to 1 if the curve is mostly straight
        if cosineAngle < 0.99:
            #the track might be curved
            ratio = abs(dist2 - dist1)/(dist2 + dist1)
            #the track might be curved if these consecutive waypoints are relatively close together
            #if dist1 and dist2 are close to the same values, ratio will be very small.
            #use a cutoff of 0.2 to decide that the angle between waypoint vectors is noticable
            #so we'll look at the relative spacing between waypoints
            if ratio < 0.2:
                #this is the only way that we will consider the track curved
                trackCurved = True 
    
    if trackCurved:
        # track is curved
        if distance_from_center <= marker_2:
            reward += 5 + (500 * max(progress, .01))/(steps**2 + 1)**.5
        elif distance_from_center <= marker_3:
            reward += 3 + (500 * max(progress, .01))/(steps**2 + 1)**.5
    else:
        #assume track is straight
        reward += (speed**3)/3.75 + (900 * max(progress, .01))/(steps**2 + 1)**.5
   
    if is_left_of_center:
        #throw in something if the car is left of center, just to see if we can
        #reinforce that lane position...it should just be a little reward, especially
        #when compared to the reward for speed and steps
        reward += 1
    
    return float(reward)
